# Machine Learning: Home Work 1
<b>Leonardo Serilli</b>
- matricola 274426
- 25/11/2020

In [11]:
import pandas as pd
import numpy as np
import sys 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

## Read Data
Here the program read the **Letters.csv** from the file system.
It also extract his **features** and, for each of it, his **alphabet**; those will be used for the **One-hot encoding** as described below.

NOTE_2: As asked in **point 6 of the homework** you can choose the Data Set just changing the name in $pd.read\_csv()$ function. 

In [12]:
#NOTE: you can change 'pd.read_csv()' argument and re-run just 
 # this and the last jupyer notebook blocks to try a new experiment on a different Data Set
DATA = pd.read_csv('Letters.csv')
features = [i for i in DATA]
alphabets = [DATA[i].unique().tolist() for i in DATA]

### One-Hot encoding
I have first defined a way to encode my Data Set to satisfy the **point 1 of the homework**:
- **encode(x,y)**: will return the index of the value '1' for the one-hot encoding of the value of x upon the alphabet of the feature y;
- **encodeDS(DS)**: relying on the above function, will return the encoded version of the Data Set, in wich, all elements will be substitued by the index of '1' for his one-hot encoding.

##### starting Data Set (head(10))
<img src="DATA.png" />

##### one-hot encoded Data Set (head(5))
<img src="df-charVec.png" />

##### one-hot index encoded Data Set (head(10))
<img src="df_index_charVec.png" />


In [13]:
# return the encoded version of the value of x under the feature y 
def encode(x,y):
    charVect = []
    for i in range(len(alphabets[y])):
        if(alphabets[y][i]!=x):
            charVect.append(0)
        else:
            charVect.append(1)
    # notice here that without the 'index()' it would return the Characteristic Vector itself
    return charVect.index(1) 

def encodeDS(DS):
    encDS = []
    for index, row in DATA.iterrows():      
        encObservable = []
        for i in range(len(row)):
            encObservable.append(encode(row[i], features.index(row.index[i])))
        encDS.append(encObservable)
    return encDS

## Splitting Data Set into Train and Test set for each possible output feature
In order to satisfy **point 2 of the homework**, I have defined a **${TrainSet}$** for each feature.<br>
In this way I'll be able to **build a model for every possible missing value** in the testSet.
- **multiSplit_df(df)**: takes in input a Data Frame and, ${\forall {f}\in features }$ it build ${TS_f}$ in which ${f}$ is the output feature. It also build a single ${TestSet}$ without choosing the output feature, this will be done later.

In order to satisfy the **point 3 of the homework**, I have defined a function to indroduce missing values (*NaNs*) in our Test Set (obtained by the **$multiSplit\_df(df)$** function), and also a way to deal with those introduced missing values:
- **multi_nan_intro(x_test, nan_ratio)**: takes the input Test Set ($x_test$) and a percentage ($nan\_ratio$) that indicates how many missing values to indroduce in it;
- **multi_nan_subs(nan_x_test)**: takes advantage of the function **fillna()** to substitute every missing value it finds with the **mean value (mean() function)** of the feature in which the *NaN* has been found.

NOTE: the way in wich models deal with missing values in prediction can be modified just changing the $fillna()$ argument in the $nan\_subs()$ function.

##### Test Set (head(10)) after missing values introduction (with a 20% ratio)
<img src="Nan_intro.png" />

##### Test Set (head(10)) after missing values substitution
<img src="Nan_subs.png" />

In [14]:
#NOTE: you can change 'fillna()' argument and re-run just 
 # this and the last jupyer notebook blocks to try a new experiment on the same Dataset
  # with a different approach to missing values in predictions

# split the given Data Set in n TS, one for each feature (that is, possible output value), 
 # and a single testSet, not defining it's output feature (it is done later)
def multiSplit_df(df):
    x_trains, x_tests,y_trains, y_tests = [], [], [], []

    for i in range(len(list(df.columns))):
        t_df = df
        y = t_df[i]
        x = t_df.drop([i], axis=1)
        
        x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.20, random_state=30)

        x_trains.append(x_train)
        y_trains.append(y_train)
        x_tests.append(x_test)
        y_tests.append(y_test)
        
    x_tests[0][0] = y_tests[0]
    
    return x_trains, y_trains, x_tests[0]

# introduces missing values in every test set in input, according to the percentage indicated by 'nan_ratio'
def nan_intro(x_test, nan_ratio):
    if(nan_ratio>=1):
        sys.exit("nan ratio must be at most 1")
    nan_mat = np.random.random(x_test.shape) < nan_ratio 
    x_test = x_test.mask(nan_mat)
    return x_test

# substitute each missing value in the given test sets with the mean value of the feature where the NaN has been found
# NOTE: the way in wich models deal with missing values in prediction can be modified 
 # just changing the $fillna()$ argument
def nan_subs(nan_x_test):
    nan_x_test = pd.DataFrame(nan_x_test).fillna(nan_x_test.mean().astype(int))
    return nan_x_test

## Building Models
Here, firstly i have created, for every possibile output, a $TS=\{x\_trains,\space y\_trains\}$, and a single  $testSet=\{x\_tests,\space y\_tests\}$.

Those are created relying on the **multiSplit_df()** function described above.

Train Sets have been used then to **build and train multiple MLP classifiers**, each able to predict their associated output feature.

In [15]:
# encode our DS into One-Hot encoded one
df = (pd.DataFrame(encodeDS(DATA)))

TS, testSet = {}, {}
TS["x_trains"], TS["y_trains"]  = [], []
testSet["x_test"], testSet["y_test"] = [], []

# create out Train Sets and a single Test Set
TS["x_trains"], TS["y_trains"], testSet["x_test"] = multiSplit_df(df)

In [6]:
# train our models, one for each feature
models = [MLPClassifier(hidden_layer_sizes=(17,17,17), activation='relu', solver='adam', max_iter=1000) for i in range(len(list(df.columns)))]
for i in range(len(models)):
    models[i].fit(TS["x_trains"][i],TS["y_trains"][i]) 

## Predictions with Nan Values
Here, to satisfy **points 3 and 4 of the homework**, firstly i define a set of ratios that define the percentage of missing values to introduce, into the Test Set, for each run of the experiment. 

Then, for every ratio, the program:
- (1) Introduces missing values (*Nan*) in the Test Set according to the ratio;
- (2) For each feature (column) $f$ of the test set:
<ul>
    <li>Set $f$ as <b>output of the TestSet</b> ($testSet["y\_test"]$) removing it from the <b>TestSet's input part</b> ($testSet["x\_test"]$).</li>
    
    <li>For each sample (row) $s$ in the Test Set, <b>if it has a missing value as output</b>, we substitute it with it's <b>prediction</b> $prediction(s)$ (computed by the model previusly created for that feature).</li>
    
    <li><b>Stores the accurancy score</b> for the current feature $f$, in an array ($acc\_scores[]$) .</li>
    <li><b>Insert back</b> in the in the Test Set's input part ($testSet["x\_test"]$), the predicted column ($testSet["y\_test"]$).</li>
</ul>

NOTE_1: Every iteration of (2) has more accurancy than the previous one, beacause less features in the TestSet will have missing values.

NOTE_2: As asked in **point 6 of the homework** you can choose feature to be predicted just by changing indexes of feature in the $fs\_to\_predict$ array.


In [21]:
#NOTE: you can change 'nan_ratios' and 'fs_to_predict' variables and re-run just 
 # this jupyer notebook block to try a new experiment on the same Dataset

# define missing values ratios and features to be predicted in the experiment
nan_ratios = [0.0, 0.1, 0.2, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
fs_to_predict = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

for ratio in nan_ratios:
    acc_scores = []
    # noNan_TestSet is used as expected outputs of each feature
    noNan_TestSet = testSet["x_test"]
    # Introduce missing values in the TestSet with nan_intro() function
    testSet["x_test"] = nan_intro(testSet["x_test"], ratio)

    # For any column (features) of Test Set
    for i in range(len(testSet["x_test"].columns)):
        if i in fs_to_predict:
            # Take a column (feature) to be considered as output of the TestSet 
            testSet["y_test"] = testSet["x_test"][i]
            testSet["x_test"] = testSet["x_test"].drop([i], axis=1)

            # Stores predictions made by the model for the current feature;
             # predictions are made approximating, by nan_subs() function,  missing values in the input part of TestSet 
            predictions = models[i].predict(nan_subs(testSet["x_test"]))
            k = 0
            # Cicle through outputs of the Test Set
            for j, row in testSet["y_test"].iteritems():
                if(np.isnan(row)):
                    # if a sample's output is missing, we substitute it with the predicted output for it
                    testSet["y_test"][j] = predictions[k]
                k += 1

            # Stores the accurancy of the prediction comparing:
             # the TestSet column before the missing value introduction with
              # the column predicted after the missing value introduction
            acc_scores.append(accuracy_score(noNan_TestSet[i], testSet["y_test"]))

            # the predicted column is put back in the input part of the TestSet;
             # NOTE: every iteration has more accurancy than the previous one, beacause less columns
                # in the TestSet will have missing values
            testSet["x_test"][i] = testSet["y_test"]

    print("\n_______ Accurancy with missing value ratio =", ratio, "_______\n")
    for i in range(len(acc_scores)):
        if i in fs_to_predict:
            print("\t\t",features[i], "\t=", "{:.2f}".format(acc_scores[i]))


_______ Accurancy with missing value ratio = 0.0 _______

		 lettr 	= 1.00
		 xbox 	= 1.00
		 ybox 	= 1.00
		 width 	= 1.00
		 high 	= 1.00
		 onpix 	= 1.00
		 xbar 	= 1.00
		 ybar 	= 1.00
		 x2bar 	= 1.00
		 y2bar 	= 1.00
		 xybar 	= 1.00
		 x2ybr 	= 1.00
		 xy2br 	= 1.00
		 xege 	= 1.00
		 xegvy 	= 1.00
		 yege 	= 1.00
		 yegvx 	= 1.00

_______ Accurancy with missing value ratio = 0.1 _______

		 lettr 	= 0.91
		 xbox 	= 0.94
		 ybox 	= 0.92
		 width 	= 0.96
		 high 	= 0.97
		 onpix 	= 0.92
		 xbar 	= 0.94
		 ybar 	= 0.95
		 x2bar 	= 0.94
		 y2bar 	= 0.93
		 xybar 	= 0.96
		 x2ybr 	= 0.98
		 xy2br 	= 0.94
		 xege 	= 0.94
		 xegvy 	= 0.94
		 yege 	= 0.96
		 yegvx 	= 0.98

_______ Accurancy with missing value ratio = 0.2 _______

		 lettr 	= 0.82
		 xbox 	= 0.88
		 ybox 	= 0.87
		 width 	= 0.93
		 high 	= 0.94
		 onpix 	= 0.84
		 xbar 	= 0.90
		 ybar 	= 0.90
		 x2bar 	= 0.90
		 y2bar 	= 0.87
		 xybar 	= 0.90
		 x2ybr 	= 0.93
		 xy2br 	= 0.87
		 xege 	= 0.86
		 xegvy 	= 0.89
		 yege 	=